In [1]:
%cd /home/561/ww5368/ww5368/NL-Summ-LLM/src

/scratch/hn98/ww5368/NL-Summ-LLM/src


In [2]:
from pathlib import Path

path = Path("../../LLM4POI/datasets")
nyc_dataset_path = path / "nyc" / "preprocessed"

In [3]:
def create_checkin_prompt(row):
    prompt = 'At {UTCTimeOffset}, user {UserId} visited POI id {PoiId} which is a/an {PoiCategoryName} with category id {PoiCategoryId}.'
    return prompt.format(**row)

In [4]:
import pandas as pd

train_df = pd.read_csv(nyc_dataset_path / "train_sample.csv")
train_df.loc[:, "prompt"] = train_df.apply(create_checkin_prompt, axis=1)

In [5]:
uniq_user_df = train_df.groupby('UserId').size().reset_index(name='count')
uniq_user_df

,UserId,count
0,0,19
1,1,36
2,2,6
3,3,36
4,4,55
...,...,...
1042,1042,53
1043,1043,77
1044,1044,44
1045,1045,191


In [6]:
users = train_df["UserId"].unique()

user_id = 0
user_df = train_df[train_df["UserId"] == user_id]
user_history_prompt = " ".join(user_df["prompt"].values)
user_history_prompt

'At 2012-04-13 21:11:20, user 0 visited POI id 556 which is a/an Bar with category id 47. At 2012-04-14 00:45:13, user 0 visited POI id 256 which is a/an Bar with category id 46. At 2012-04-14 13:45:23, user 0 visited POI id 355 which is a/an Hotel with category id 258. At 2012-04-14 18:07:43, user 0 visited POI id 4180 which is a/an Bar with category id 36. At 2012-04-14 19:15:07, user 0 visited POI id 212 which is a/an Burger Joint with category id 129. At 2012-04-19 15:02:25, user 0 visited POI id 693 which is a/an Coffee Shop with category id 221. At 2012-04-19 18:51:19, user 0 visited POI id 1348 which is a/an American Restaurant with category id 101. At 2012-04-19 21:49:26, user 0 visited POI id 199 which is a/an Diner with category id 96. At 2012-04-20 20:15:18, user 0 visited POI id 534 which is a/an French Restaurant with category id 21. At 2012-04-21 12:28:11, user 0 visited POI id 4141 which is a/an American Restaurant with category id 101. At 2012-04-21 14:48:58, user 0 vis

In [7]:
from gpt import GPT
import openai

openai.api_key_path = "/home/561/ww5368/.openai_key"

llm = GPT()
result = llm.generate(user_id=user_id, user_history_prompt=user_history_prompt)

In [8]:
result

{'traits': ['extroverted',
  'agreeable',
  'conscientious',
  'emotionally stable',
  'open'],
 'attributes': ['adult', 'male', 'college & beyond', 'middle'],
 'preferences': ['socializing',
  'cuisine exploration',
  'coffee culture',
  'bar hopping'],
 'routines': ['visiting bars frequently',
  'enjoying American cuisine',
  'coffee shop visits',
  'going to the movies'],
 'user_profile': 'User 0 is a sociable adult male who enjoys exploring the culinary scene, with a frequent preference for bars and American restaurants. He often visits local coffee shops, indicating a passion for coffee culture. With an educational background of some college, he likely values social interactions and has a wide network of acquaintances. User 0 appears to regularly indulge in leisure activities, often spending time at popular venues like bars, burger joints, and diners, suggesting an outgoing and adventurous lifestyle. He enjoys a blend of casual and upscale dining experiences, reflected in his visi

In [ ]:
import json

with open("../data/nyc/user_profile_0.json", "w") as f:
    json.dump(result, f, indent=4)

In [9]:
import tiktoken
from tqdm.auto import tqdm

tokenizer = tiktoken.encoding_for_model("gpt-4o")
sum_input_tokens = 0

for user_id in tqdm(users):
    user_df = train_df[train_df["UserId"] == user_id]
    user_history_prompt = " ".join(user_df["prompt"].values)
    prompt = llm.prompt.format(user_id=user_id, user_history_prompt=user_history_prompt)

    tokens = tokenizer.encode(prompt)
    sum_input_tokens += len(tokens)

/scratch/hn98/ww5368/micromamba/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 1047/1047 [00:03<00:00, 315.32it/s]


In [10]:
sum_input_tokens / 1_000_000 * 0.15

0.50672085

In [12]:
import json

sum_output_tokens = 0

json_files = list(Path("../data/nyc").rglob("*.json"))
for f in json_files:
    with open(f, "r") as f:
        data = json.load(f)
    line = data.__repr__()
    sum_output_tokens += len(tokenizer.encode(line))

In [13]:
sum_output_tokens / len(json_files) * len(uniq_user_df) / 1_000_000 * 0.6

0.18440216756756753